In [1]:
import uuid
import os 
import json
import pandas as pd 
from collections import defaultdict
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
turk_annotations = pd.read_csv(f'{os.getcwd()}/corpus/annotations_final.csv')
print("read annot")

read annot


In [3]:
with open(f'{os.getcwd()}/corpus/corpus.json', 'r') as j:
     corpus = json.loads(j.read())

print("read json")

read json


In [4]:
annotations_texts = turk_annotations['text'].to_list()
annotations_labels = turk_annotations['label'].to_list()

In [5]:
analyzer = SentimentIntensityAnalyzer()

In [6]:
combined_corpus = []
for post in corpus:
    annotations_included = 0
    document = defaultdict(str)
    document['uuid'] = str(uuid.uuid4())

    for key, value in post.items():
        # add original key values
        if key != 'text':
            document[key] = post[key]
        else:
            document[key] = post[key].lower()

    try:
        anot_index = annotations_texts.index(post['text'])
        anot_label = annotations_labels[anot_index]
        document['turkAnnotated'] = 'True'
        document['turkAnnotation'] = str(anot_label)
        annotations_included += 1 
    except:
        document['turkAnnotated'] = 'False'
        document['turkAnnotation'] = 'N/A'

    document['vaderCompound'] = str(analyzer.polarity_scores(document['text'])['compound'])
    if float(document['vaderCompound']) >= 0:
        document['vaderSentiment'] = 'Positive'
    else:
        document['vaderSentiment'] = 'Negative'
    combined_corpus.append(document)

In [7]:
combined_corpus[8]

defaultdict(str,
            {'uuid': 'cb6b8588-1e95-4f6c-92fa-d94d3ab4096e',
             'title': 'GME 🌈🐻 Short position',
             'date': '2020-12-27 13:52:03',
             'text': "***msft***gme  short position this post is to inverse wsb , well because you know why . let ' s do some half ass  dd , maybe a bull can change my mind . facts of gme : b   ; m stores are dying and have been dying for quite some time . every video game in the very near future will be digital , thus putting less of a need for b   ; m stores . gme is the most shorted stock . gme stock was at $ 5 prior to covid and was on a crazy decline . what changed since covid hit , and bankruptcy for this company that makes it so valuable to autists . there is no trade in value for digital games . where do people think gamestop will cover revenue that can ' t be directly outsourced to a much better supplier such as msft , sony , best buy and more . gme is too much of a novelty and has continued to fail at adapting

In [8]:
with open(f'{os.getcwd()}/Corpus/annotated_corpus.json', 'w') as fout:
    json.dump(combined_corpus, fout)